# Autonomous Trading Agents - With 0xMONK from FereAI

<a target="_blank" href="https://colab.research.google.com/github/fere-ai/agentic-examples/blob/main/python/trading-agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook demonstrates the ease of building autonomous trading agents. It's super easy using the `0xMONK` APIs from FereAI.

- [Developer Docs](https://docs.fereai.xyz/docs/api/api-introduction)
- [Get Your API Key](https://docs.google.com/forms/d/e/1FAIpQLScFAWfT5u2kwKx8tQkkL9AfdP8NIlNIn6tXJUQcRipZEUZokA/viewform)
- [#dev-help on our Discord](https://discord.com/invite/3fsm5XJNW8)

In [ ]:
import requests
import json
from pprint import pprint

In [ ]:
fere_api_key = "YOUR_API_KEY" # @param {type: "string"}
fere_user_id = "YOUR_USER_ID" # @param {type: "string"}

api_headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'X-FRIDAY-KEY': fere_api_key
}

## 1. Create an Agent

Instructions Examples: https://docs.fereai.xyz/docs/product/0xMONK/#coin-decision-instructions


In [53]:
url = "https://api.fereai.xyz/ta/agent/"

name = "Disciple 421" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "trending" # @param ["trending", "latest"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = 0.2 # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = True # @param {type: "boolean"}
simulation_initial_usd = 1000 # @param {type: "number"}


In [54]:

def create_agent(
    fere_user_id,
    name,
    description,
    persona,
    data_source,
    decision_prompt_pool,
    decision_prompt_portfolio,
    twitter_username=None,
    fc_username=None,
    simulation=False,
    simulation_initial_usd=0,
    max_investment_per_session=0,
    stop_loss=0,
    trailing_stop_loss=0,
    take_profit=0
):
    """
    Sends an investment request with the given parameters to the specified URL.

    Args:
        api_headers (dict): Headers for the API request.
        fere_user_id (str): The user ID.
        name (str): The name of the investment request.
        description (str): The description of the investment request.
        persona (str): The persona for the investment.
        data_source (str): The data source for the investment.
        decision_prompt_pool (str): Decision prompt pool.
        decision_prompt_portfolio (str): Decision prompt portfolio.
        twitter_username (str, optional): Twitter username. Defaults to None.
        fc_username (str, optional): FC username. Defaults to None.
        simulation (bool, optional): Indicates whether it's a dry run. Defaults to False.
        simulation_initial_usd (float, optional): Initial USD for simulation. Defaults to 0.
        max_investment_per_session (float, optional): Max investment per session. Defaults to 0.
        stop_loss (float, optional): Stop-loss percentage. Defaults to 0.
        trailing_stop_loss (float, optional): Trailing stop-loss percentage. Defaults to 0.
        take_profit (float, optional): Take-profit percentage. Defaults to 0.

    Returns:
        dict or None: JSON response if the request succeeds, None otherwise.
    """
    payload = json.dumps({
        "user_id": fere_user_id,
        "name": name,
        "description": description,
        "persona": persona,
        "data_source": data_source,
        "decision_prompt_pool": decision_prompt_pool,
        "decision_prompt_portfolio": decision_prompt_portfolio,
        "twitter_username": twitter_username,
        "fc_username": fc_username,
        "dry_run": simulation,
        "dry_run_initial_usd": simulation_initial_usd,
        "max_investment_per_session": max_investment_per_session,
        "stop_loss": stop_loss,
        "trailing_stop_loss": trailing_stop_loss,
        "take_profit": take_profit
    })

    response = requests.request("PUT", url, headers=api_headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(response.text)
        return None

agent = create_agent(fere_user_id, name,description,persona,data_source,
             decision_prompt_pool, decision_prompt_portfolio, twitter_username,
             fc_username, simulation, simulation_initial_usd,
             max_investment_per_session, stop_loss, trailing_stop_loss,
             take_profit)

pprint(f"Agent created")
pprint(agent)

{'data_source': 'trending',
 'decision_prompt_pool': 'You are a pro memecoin trader on Solana. Use your '
                         'understanding of memcoins, market psychology and '
                         'on-chain metrics to identify coins with high growth '
                         'potentials.  MISSION: Make an informed and '
                         'calculated judgement on weather to buy, hold, sell '
                         'or pass on a memecoin.  GOAL: Your GOAL is to '
                         'generate consistent profits over a **longer-term '
                         'period (3 - 7 days)** by blending **momentum '
                         'trading, volatility management, and adaptive '
                         'feedback loops**.  STRATEGY: ### 1. **Initial '
                         'Assessment for Entry**    - **Assess Pool '
                         'Age:**      - If the pool was created recently '
                         '(within 1-2 days), it suggests early-stage mo

## 2. Get Disciple Agents from a given User

In [64]:
def fetch_disciples(fere_user_id):
    """
    Fetches a list of disciples for the given user ID.

    Args:
        fere_user_id (str): The user ID for which to fetch disciples.
        api_headers (dict): Headers for the API request.

    Returns:
        list: A list of disciples if the request succeeds, or an empty list otherwise.
    """
    url = f"https://api.fereai.xyz/ta/agent/{fere_user_id}/"
    payload = {}

    response = requests.request("GET", url, headers=api_headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        print(f"Disciples Created: {len(data)}\n")
        for disciple in data:
            print(f"Name: {disciple['name']}")
            print(f"ID: {disciple['id']}\n")
        return data
    else:
        print(f"Failed to fetch disciples: {response.text}")
        return []

fetch_disciples(fere_user_id)

Disciples Created: 4

Name: Disciple 420
ID: 22f1ce39-8520-4af0-bdbe-fdb8b7d21bf0

Name: Disciple 420
ID: 403a7be0-ab41-4532-a41e-a4eaed8b93b3

Name: Disciple 420
ID: fa0808bc-41b7-427d-b790-d84bfbec94b0

Name: Disciple 421
ID: 14def492-9fbd-4cad-a331-dcaaca5d8f01



## 3. Get portfolio

In [74]:
disciple_agent_id = "14def492-9fbd-4cad-a331-dcaaca5d8f01" # @param {type: "string"}

In [75]:
def get_portfolio(disciple_agent_id):
  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/portfolio/"

  payload = {}
  response = requests.request("GET", url, headers=api_headers, data=payload)

  if response.status_code == 200:
    data = response.json()
    return data
  else:
    print(response.text)


get_portfolio(disciple_agent_id)

{'id': 'baab2eec-3a05-49a1-bc7e-b994f9615347',
 'agent_id': '14def492-9fbd-4cad-a331-dcaaca5d8f01',
 'start_time': '2024-11-26T09:34:15.712055Z',
 'start_usd': 1000.0,
 'start_native': 4.24808836023789,
 'curr_realised_usd': 980.073549705943,
 'curr_realised_native': 4.16298788334761,
 'curr_unrealised_usd': 19.465008787523,
 'curr_unrealised_native': 0.0841309501410071,
 'dry_run': True}

## 4. Get Holdings

In [85]:
def get_holdings(disciple_agent_id):
  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/holdings/"

  payload = {}

  response = requests.request("GET", url, headers=api_headers, data=payload)
  if response.status_code == 200:
    data = response.json()
    return data
  else:
    print(response.text)

holdings = get_holdings(disciple_agent_id)

print(f"Total holdings: {len(holdings)}")

pprint(holdings)

Total holdings: 1
[{'agent_id': '14def492-9fbd-4cad-a331-dcaaca5d8f01',
  'base_address': 'GqmEdRD3zGUZdYPeuDeXxCc8Cj1DBmGSYK97TCwSpump',
  'bought_at': '2024-11-26T10:05:45.052557Z',
  'buying_price_native': 4.30841681980038e-05,
  'buying_price_usd': 0.0100882458880364,
  'curr_price_native': 4.17057630968669e-05,
  'curr_price_usd': 0.0096884,
  'decimals': 6,
  'dry_run': True,
  'id': '85515f1b-6269-4dd6-a2e1-d6ee7d668003',
  'is_active': True,
  'pool_address': 'GvDBhjocyfDZuGWqkm4ZU89W2YXaVLadWDUXeJHr9h8s',
  'pool_name': 'e/acc / SOL',
  'profit_abs_native': -0.00272264584326314,
  'profit_abs_usd': -0.789781425003625,
  'profit_per_native': -3.19933088832562,
  'profit_per_usd': -3.96348277464743,
  'token_name': 'Effective accelerationism',
  'tokens_bought': 1975.214573}]


## 5. Get Trades


In [77]:
def get_trades(disciple_agent_id):
  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/trades/"

  payload = {}

  response = requests.request("GET", url, headers=api_headers, data=payload)

  if response.status_code == 200:
    return response.json()
  else:
    pprint(response.text)

get_trades(disciple_agent_id)

[{'created_at': '2024-11-26T10:05:43Z',
  'agent_id': '14def492-9fbd-4cad-a331-dcaaca5d8f01',
  'base_address': 'GqmEdRD3zGUZdYPeuDeXxCc8Cj1DBmGSYK97TCwSpump',
  'pool_name': 'e/acc / SOL',
  'decision': 1,
  'price_usd': 0.0100882458880364,
  'price_sol': 4.30841681980038e-05,
  'in_amount': 84961767.0,
  'out_amount': 1975214573.0,
  'gas_fee': 127000.0,
  'jito_fee': 32101.0,
  'other_amount_threshold': 1965338501.0,
  'reason': 'The pool has shown a consistent upward trend in the past 24 hours, with a price change percentage of 8.69% in the last hour and 16.58% in the last 6 hours. The volume has also been increasing, with a volume of 75920.31347229001 in the last hour and 1251851.08394443 in the last 24 hours. The OHLCV data also shows a bullish trend, with the price closing above the 12-hour moving average. Therefore, it is recommended to buy this pool.',
  'future_action': "Monitor the pool's performance and adjust the position size based on the momentum. If the price continues 

## 6. Get Optimal gains

In [79]:
def get_optimal_gains(disciple_agent_id):
  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/buy/"

  payload = {}
  response = requests.request("GET", url, headers=api_headers, data=payload)
  if response.status_code == 200:
    return response.json()
  else:
    print(response.txt)

get_optimal_gains(disciple_agent_id)

[{'pool_address': 'GvDBhjocyfDZuGWqkm4ZU89W2YXaVLadWDUXeJHr9h8s',
  'base_address': 'GqmEdRD3zGUZdYPeuDeXxCc8Cj1DBmGSYK97TCwSpump',
  'token_name': 'e/acc / SOL',
  'profile_pic': 'https://coin-images.coingecko.com/coins/images/50905/large/GqmEdRD3zGUZdYPeuDeXxCc8Cj1DBmGSYK97TCwSpump.webp?1729520765',
  'peaked_at': '2024-11-26T10:05:45.214017+00:00',
  'notice_price_native': 4.30841681980038e-05,
  'price_native': 4.30841681980038e-05,
  'profit_per_native': 0.0,
  'agent_id': '14def492-9fbd-4cad-a331-dcaaca5d8f01',
  'id': 'fd52730b-89ad-438c-ba99-7c9d422ddd6c',
  'pool_name': 'Effective accelerationism',
  'symbol': 'e/acc',
  'noticed_at': '2024-11-26T10:05:45.214017+00:00',
  'notice_price_usd': 0.0100882458880364,
  'price_usd': 0.0100882458880364,
  'profit_per_usd': 0.0,
  'dry_run': True}]

## 7. Sell a holding manually

In [ ]:
holding_id = "" # @param {type: "string"}
quantity = "all" # @param {type: "string"}

A sell is executed as a background task. Upon calling the sell API, it returns a task ID. This task ID will be used to check the task status.

In [ ]:

def sell_holding(disciple_agent_id, holding_id, quantity):

  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/sell/f{holding_id}/f{quantity}/"
  payload = {}

  response = requests.request("POST", url, headers=api_headers, data=payload)

  if response.status_code == 200:
    return response.json()
  else:
    print(response.text)

task = sell_holding(disciple_agent_id, holding_id, quantity)

pprint(task)

Now once we have the task ID, we can check the status of the task.

In [ ]:
task_id = "all" # @param {type: "string"}

url = f"https://api.fereai.xyz/ta/task/status/{task_id}/"

payload = {}

response = requests.request("GET", url, headers=api_headers, data=payload)

print(response.json())

Depending on the status, show the updates to user.

# Disciple Lifecycle APIs

## 8. Update a Disciple

In [ ]:
disciple_id = "" # @param {type: "string"}
name = "Disciple 420" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "trending" # @param ["trending", "latest"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = 0.2 # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = True # @param {type: "boolean"}
simulation_initial_usd = 1000 # @param {type: "number"}

In [ ]:

url = f"https://api.fereai.xyz/ta/agent/{disciple_id}/"

payload = json.dumps({
  "user_id": fere_user_id,
  "name": name,
  "description": description,
  "persona": persona,
  "data_source": data_source,
  "decision_prompt_pool": decision_prompt_pool,
  "decision_prompt_portfolio": decision_prompt_portfolio,
  "twitter_username": twitter_username,
  "fc_username": fc_username,
  "dry_run": simulation,
  "dry_run_initial_usd": simulation_initial_usd,
  "max_investment_per_session": max_investment_per_session,
  "stop_loss": stop_loss,
  "trailing_stop_loss": trailing_stop_loss,
  "take_profit": take_profit
})

response = requests.request("PATCH", url, headers=api_headers, data=payload)

print(response.json())

## 9. Delete the Agent

In [ ]:
def delete_disciple(disciple_agent_id):
  url = f"https://api.fereai.xyz/ta/agent/{disciple_agent_id}/"

  payload = {}
  response = requests.request("DELETE", url, headers=api_headers, data=payload)

  if response.status_code == 200:
    return response.json()
  else:
    print(response.text)

delete_disciple(disciple_agent_id)